In [62]:
import itertools

In [63]:
FDIC = pd.DataFrame(pd.read_csv("ACdata.csv"))
print(FDIC.shape)
FDIC['state_abv'] = list(map(lambda x: x[-2:], FDIC['county_state']))
#states = ["AL","AK","FL","GA","LA","MD","MS","NC","SC","TN","TX"]
#FDIC = FDIC[FDIC['state_abv'].isin(states)]
print(FDIC.shape)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(653106, 40)
(653106, 41)


In [64]:
FDIC.columns

Index(['FIPS_county', 'year', 'cert', 'county_state', 'state', 'FIPS_state',
       'pop', 'bankname', 'cpi_2018', 'deposits', 'branch_count', 'cb',
       'asset', 'all_loans', 'equity', 'real_estate_loans', 'ag_loans',
       'ci_loans', 'consumer_loans', 'credit_card_loans', 'na_asset',
       'na_real_estate_loans', 'na_ag_loans', 'na_ci_loans',
       'na_consumer_loans', 'na_credit_card_loans',
       'dollarloans_nonfarm_bus_LT1M', 'dollarloans_ci_LT1M',
       'dollarloans_ag_bus_LT500K', 'numloans_nonfarm_bus_LT1M',
       'numloans_ci_LT1M', 'numloans_ag_bus_LT500K', 'percap_pers_inc', 'jobs',
       'farm_inc', 'farm_jobs', 'nonfarm_inc', 'nonfarm_jobs', 'owner_inc',
       'owner_jobs', 'state_abv'],
      dtype='object')

In [65]:
sb_cols = ['FIPS_county', 'county_state', 'year', 'pop', 'cpi_2018', 'cb', 'branch_count', 'deposits', 'asset', 'all_loans', 'equity',
           'ci_loans', 'credit_card_loans', 'na_asset', 'na_ci_loans', 'na_credit_card_loans',
           'dollarloans_nonfarm_bus_LT1M', 'dollarloans_ci_LT1M','numloans_nonfarm_bus_LT1M',
           'numloans_ci_LT1M', 'percap_pers_inc', 'jobs','farm_inc', 'farm_jobs', 'nonfarm_inc', 'nonfarm_jobs', 'owner_inc',
       'owner_jobs']
FDIC_sb = FDIC[sb_cols]
FDIC_sb.head()

,FIPS_county,county_state,year,pop,cpi_2018,cb,branch_count,deposits,asset,all_loans,...,numloans_nonfarm_bus_LT1M,numloans_ci_LT1M,percap_pers_inc,jobs,farm_inc,farm_jobs,nonfarm_inc,nonfarm_jobs,owner_inc,owner_jobs
0,1001,"Autauga, AL",1994,38186.0,58.883015,1,3,10861,288171,141793,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0
1,1001,"Autauga, AL",1994,38186.0,58.883015,0,1,27982,9709521,5933900,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0
2,1001,"Autauga, AL",1994,38186.0,58.883015,0,2,23991,8219440,5740534,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0
3,1001,"Autauga, AL",1994,38186.0,58.883015,1,3,121131,160392,87651,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0
4,1001,"Autauga, AL",1994,38186.0,58.883015,0,2,17250,499998,337683,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0


In [66]:
FDIC_sb_i = FDIC_sb.reset_index(drop = True)

In [67]:
print("Number of missing values in na_credit_card_loans column is", sum(FDIC_sb['na_credit_card_loans'].isna()))
print("Number of missing values in dollarloans_nonfarm_bus_LT1M column is", sum(FDIC_sb['dollarloans_nonfarm_bus_LT1M'].isna()))
print("Number of missing values in dollarloans_ci_LT1M column is", sum(FDIC_sb['dollarloans_ci_LT1M'].isna()))
print("Number of missing values in numloans_nonfarm_bus_LT1M column is", sum(FDIC_sb['numloans_nonfarm_bus_LT1M'].isna()))
print("Number of missing values in numloans_ci_LT1M column is", sum(FDIC_sb['numloans_ci_LT1M'].isna()))

Number of missing values in na_credit_card_loans column is 12798
Number of missing values in dollarloans_nonfarm_bus_LT1M column is 97376
Number of missing values in dollarloans_ci_LT1M column is 97376
Number of missing values in numloans_nonfarm_bus_LT1M column is 97376
Number of missing values in numloans_ci_LT1M column is 97376


In [68]:
#eliminating null values, adjusting income 
FDIC_sb_i = FDIC_sb_i[~FDIC_sb_i['na_credit_card_loans'].isna()].fillna(0)
FDIC_sb_i = FDIC_sb_i[~FDIC_sb_i['dollarloans_ci_LT1M'].isna()].fillna(0)

In [69]:
keep = ["FIPS_county", "year", "pop", "cpi_2018","jobs", "percap_pers_inc", "cb", ]
sums = ["branch_count",'deposits', 'asset', 'all_loans', 'equity',
           'ci_loans', 'credit_card_loans', 'na_asset', 'na_ci_loans', 'na_credit_card_loans',
           'dollarloans_nonfarm_bus_LT1M', 'dollarloans_ci_LT1M','numloans_nonfarm_bus_LT1M',
           'numloans_ci_LT1M','farm_inc', 'farm_jobs', 'nonfarm_inc', 'nonfarm_jobs', 'owner_inc',
       'owner_jobs']

FDIC_sb_gi = FDIC_sb_i.groupby(keep)[sums].sum()
print(FDIC_sb_gi.shape)
FDIC_sb_gi

(134352, 20)


branch_count  \
FIPS_county year pop     cpi_2018   jobs    percap_pers_inc cb                 
1001        1994 38186.0 58.883015  12751.0 18683.0         0              5   
                                                            1              6   
            1995 39112.0 60.674587  13033.0 19350.0         0              5   
                                                            1              6   
            1996 40207.0 62.386536  13644.0 20155.0         0              5   
...                                                                      ...   
56045       2016 7213.0  95.607864  4043.0  42585.0         1              2   
            2017 6986.0  97.214310  3964.0  42816.0         0              1   
                                                            1              2   
            2018 6967.0  100.000000 4043.0  44737.0         0              1   
                                                            1              2   

                                                                deposits  \
FIPS_county year pop     cpi_2018   jobs    percap_pers_inc cb             
1001        1994 38186.0 58.883015  12751.0 18683.0         0      69223   
                                                            1     131992   
            1995 39112.0 60.674587  13033.0 19350.0         0      88316   
                                                            1     169776   
            1996 40207.0 62.386536  13644.0 20155.0         0     101855   
...                                                                  ...   
56045       2016 7213.0  95.607864  4043.0  42585.0         1     152963   
            2017 6986.0  97.214310  3964.0  42816.0         0      66557   
                                                            1     149275   
            2018 6967.0  100.000000 4043.0  44737.0         0      71540   
                                                            1     144001   

                                                                   asset  \
FIPS_county year pop     cpi_2018   jobs    percap_pers_inc cb             
1001        1994 38186.0 58.883015  12751.0 18683.0         0   18428959   
                                                            1     448563   
            1995 39112.0 60.674587  13033.0 19350.0         0   32794998   
                                                            1     446128   
            1996 40207.0 62.386536  13644.0 20155.0         0   36609731   
...                                                                  ...   
56045       2016 7213.0  95.607864  4043.0  42585.0         1     397900   
            2017 6986.0  97.214310  3964.0  42816.0         0     777479   
                                                            1     413931   
            2018 6967.0  100.000000 4043.0  44737.0         0     836692   
                                                            1     444261   

                                                                all_loans  \
FIPS_county year pop     cpi_2018   jobs    percap_pers_inc cb              
1001        1994 38186.0 58.883015  12751.0 18683.0         0    12012117   
                                                            1      229444   
            1995 39112.0 60.674587  13033.0 19350.0         0    23395196   
                                                            1      257205   
            1996 40207.0 62.386536  13644.0 20155.0         0    25758406   
...                                                                   ...   
56045       2016 7213.0  95.607864  4043.0  42585.0         1      202250   
            2017 6986.0  97.214310  3964.0  42816.0         0      623355   
                                                            1      236390   
            2018 6967.0  100.000000 4043.0  44737.0         0      672740   
                                                            1      272249   

                                           

In [70]:
indices = list(FDIC_sb_gi.index)

In [71]:
#filtering for counties that do not have 21 years of data for both community banks and non community banks
f_index = 0
cb_index = -1

county = list(map(lambda x: x[f_index], FDIC_sb_gi.index))
cb_presence = list(map(lambda x: x[cb_index], FDIC_sb_gi.index))
df = pd.DataFrame(county)
df['1'] = cb_presence
df = df.groupby([0]).mean()
desirable_c = list(df[df["1"] == .5].index)
newIndex = [x for x in indices if x[f_index] in desirable_c]
FDIC_sb_g = FDIC_sb_gi.loc[newIndex]

percap_pers_inc = list(map(lambda x: x[-2], FDIC_sb_g.index))
FDIC_sb_g["percap_pers_inc"] = percap_pers_inc
FDIC_sb_g = FDIC_sb_g.reset_index(level = -2, drop = True)

cb = list(map(lambda x: x[-1], FDIC_sb_g.index))
FDIC_sb_g["cb"] = cb
FDIC_sb_g = FDIC_sb_g.reset_index(level = -1, drop = True)

pop = list(map(lambda x: x[2], FDIC_sb_g.index))
FDIC_sb_g["pop"] = pop
FDIC_sb_g = FDIC_sb_g.reset_index(level = 2, drop = True)

<IPython.core.display.Javascript object>

In [72]:
FDIC_sb_g = FDIC_sb_g.reset_index([0, 1, 2, 3])

In [73]:
FDIC_sb_o = FDIC_sb_g.set_index(["year","cpi_2018", "FIPS_county", "pop", "percap_pers_inc", "jobs","cb"]).sort_index(level = 0)
FDIC_sb_o

branch_count  \
year cpi_2018   FIPS_county pop      percap_pers_inc jobs    cb                 
1994 58.883015  1001        38186.0  18683.0         12751.0 0              5   
                                                             1              6   
                1003        116565.0 20129.0         50770.0 0             36   
                                                             1              7   
                1005        27751.0  15930.0         12994.0 0              3   
...                                                                       ...   
2018 100.000000 56041       20299.0  40280.0         11857.0 1              2   
                56043       7885.0   48184.0         5320.0  0              5   
                                                             1              2   
                56045       6967.0   44737.0         4043.0  0              1   
                                                             1              2   

                                                                 deposits  \
year cpi_2018   FIPS_county pop      percap_pers_inc jobs    cb             
1994 58.883015  1001        38186.0  18683.0         12751.0 0      69223   
                                                             1     131992   
                1003        116565.0 20129.0         50770.0 0     978407   
                                                             1      95509   
                1005        27751.0  15930.0         12994.0 0      48752   
...                                                                   ...   
2018 100.000000 56041       20299.0  40280.0         11857.0 1     177597   
                56043       7885.0   48184.0         5320.0  0     139289   
                                                             1      96219   
                56045       6967.0   44737.0         4043.0  0      71540   
                                                             1     144001   

                                                                     asset  \
year cpi_2018   FIPS_county pop      percap_pers_inc jobs    cb              
1994 58.883015  1001        38186.0  18683.0         12751.0 0    18428959   
                                                             1      448563   
                1003        116565.0 20129.0         50770.0 0    25949612   
                                                             1      104850   
                1005        27751.0  15930.0         12994.0 0     7796559   
...                                                                    ...   
2018 100.000000 56041       20299.0  40280.0         11857.0 1      191495   
                56043       7885.0   48184.0         5320.0  0   545996109   
                                                             1      574509   
                56045       6967.0   44737.0         4043.0  0      836692   
                                                             1      444261   

                                                                 all_loans  \
year cpi_2018   FIPS_county pop      percap_pers_inc jobs    cb              
1994 58.883015  1001        38186.0  18683.0         12751.0 0    12012117   
                                                             1      229444   
                1003        116565.0 20129.0         50770.0 0    17124846   
                                                             1       59763   
                1005        27751.0  15930.0         12994.0 0     5327730   
...                                                                    ...   
2018 100.000000 56041       20299.0  40280.0         11857.0 1       58503   
                56043       7885.0   48184.0         5320.0  0   342015041   
                                                             1      312875   
                56045       6967.0   44737.0         4043.0  0      672740   
                                                          

In [74]:
"""def createDifferences(f, col, n):
        df0 = FDIC_sb_g.loc[f][::2]
        df1 = FDIC_sb_g.loc[f][1::2]
        res0 = 100 * np.diff(df0[col], n)/df0[col][:-n]
        res1 = 100 * np.diff(df1[col], n)/df1[col][:-n]
        result0 = list(itertools.chain([np.nan] * n, res0))
        result1 = list(itertools.chain([np.nan] * n, res1))
        result = [None]*(len(result0)+len(result1))
        result[::2] = result0
        result[1::2] = result1
        return result

def createTimeDif(county, column, n):
    listLoanChange = [createDifferences(f, column, n) for f in county]
    return listLoanChange"""

'def createDifferences(f, col, n):\n        df0 = FDIC_sb_g.loc[f][::2]\n        df1 = FDIC_sb_g.loc[f][1::2]\n        res0 = 100 * np.diff(df0[col], n)/df0[col][:-n]\n        res1 = 100 * np.diff(df1[col], n)/df1[col][:-n]\n        result0 = list(itertools.chain([np.nan] * n, res0))\n        result1 = list(itertools.chain([np.nan] * n, res1))\n        result = [None]*(len(result0)+len(result1))\n        result[::2] = result0\n        result[1::2] = result1\n        return result\n\ndef createTimeDif(county, column, n):\n    listLoanChange = [createDifferences(f, column, n) for f in county]\n    return listLoanChange'

In [75]:
"""#Income, pop changes same regardless of cb/non cb bank presence
FDIC_sb_g['loanChange'] = list(itertools.chain.from_iterable(createTimeDif(desirable_c,'dollarloans_ci_LT1M', 1)))
FDIC_sb_g['incomeChange'] = list(itertools.chain.from_iterable(createTimeDif(desirable_c,'percap_pers_inc', 1)))
FDIC_sb_g['popChange'] = list(itertools.chain.from_iterable(createTimeDif(desirable_c,'pop', 1)))"""

"#Income, pop changes same regardless of cb/non cb bank presence\nFDIC_sb_g['loanChange'] = list(itertools.chain.from_iterable(createTimeDif(desirable_c,'dollarloans_ci_LT1M', 1)))\nFDIC_sb_g['incomeChange'] = list(itertools.chain.from_iterable(createTimeDif(desirable_c,'percap_pers_inc', 1)))\nFDIC_sb_g['popChange'] = list(itertools.chain.from_iterable(createTimeDif(desirable_c,'pop', 1)))"

In [76]:
#do moving averages some other time

In [77]:
"""def createMVA(n, f, col):
    window_size = n
    df0 = FDIC_sb_g.loc[f][::2][col]
    df1 = FDIC_sb_g.loc[f][1::2][col]
    numbers_series0 = pd.Series(df0)
    numbers_series1 = pd.Series(df1)
    windows0 = numbers_series0.rolling(window_size)
    windows1 = numbers_series1.rolling(window_size)
    moving_averages0 = windows0.mean()
    moving_averages_list0 = moving_averages0.tolist()
    moving_averages1 = windows1.mean()
    moving_averages_list1 = moving_averages1.tolist()
    result = [None]*(len(moving_averages_list1)+len(moving_averages_list0))
    result[::2] = moving_averages_list0
    result[1::2] = moving_averages_list1  
    return result

def createMVAs(n, county, col):
    listLoanChange = [createMVA(n, f, col) for f in county]
    return listLoanChange

FDIC_sb_g['loanMovingAverage3'] = list(itertools.chain.from_iterable(createMVAs(3, desirable_c, "dollarloans_ci_LT1M_adj")))
FDIC_sb_g['loanMovingAverage5'] = list(itertools.chain.from_iterable(createMVAs(5, desirable_c, "dollarloans_ci_LT1M_adj")))
FDIC_sb_g['loanChangeMovingAverage3'] = list(itertools.chain.from_iterable(createMVAs(3, desirable_c, "loanChange")))
FDIC_sb_g['loanChangeMovingAverage5'] = list(itertools.chain.from_iterable(createMVAs(5, desirable_c, "loanChange")))
#groupedSectors['businessChangeMVA5'] = list(itertools.chain.from_iterable(createMVAs(5, counties)))"""

'def createMVA(n, f, col):\n    window_size = n\n    df0 = FDIC_sb_g.loc[f][::2][col]\n    df1 = FDIC_sb_g.loc[f][1::2][col]\n    numbers_series0 = pd.Series(df0)\n    numbers_series1 = pd.Series(df1)\n    windows0 = numbers_series0.rolling(window_size)\n    windows1 = numbers_series1.rolling(window_size)\n    moving_averages0 = windows0.mean()\n    moving_averages_list0 = moving_averages0.tolist()\n    moving_averages1 = windows1.mean()\n    moving_averages_list1 = moving_averages1.tolist()\n    result = [None]*(len(moving_averages_list1)+len(moving_averages_list0))\n    result[::2] = moving_averages_list0\n    result[1::2] = moving_averages_list1  \n    return result\n\ndef createMVAs(n, county, col):\n    listLoanChange = [createMVA(n, f, col) for f in county]\n    return listLoanChange\n\nFDIC_sb_g[\'loanMovingAverage3\'] = list(itertools.chain.from_iterable(createMVAs(3, desirable_c, "dollarloans_ci_LT1M_adj")))\nFDIC_sb_g[\'loanMovingAverage5\'] = list(itertools.chain.from_iterab

In [78]:
FDIC_sb_g.to_csv("Changes1.csv")

In [79]:
FDIC_p = FDIC.groupby(['FIPS_county', 'county_state','year', 'cpi_2018'])['percap_pers_inc', 'jobs','nonfarm_inc', 'nonfarm_jobs', 'owner_inc','owner_jobs', 'cb', 'pop'].mean()

In [80]:
FDIC_no_cb = FDIC_p[FDIC_p['cb'] == 0]

In [81]:
problematic_c = np.unique(list(map(lambda x: x[0], FDIC_no_cb.index)))

<IPython.core.display.Javascript object>

In [82]:
problematic_c

array([ 1029,  1129,  2013,  2016,  2060,  2070,  2180,  2185,  2188,
        2230,  2232,  2240,  2282,  4001,  4003,  4005,  4007,  4009,
        4011,  4012,  4017,  4023,  5029,  5129,  6009,  6011,  6015,
        6021,  6025,  6031,  6043,  6047,  6049,  6051,  6069,  6091,
        6105,  6115,  8021,  8023,  8033,  8039,  8047,  8053,  8093,
        8109, 12003, 12013, 12035, 12037, 12039, 12045, 12077, 12089,
       12093, 12107, 12129, 12133, 13011, 13053, 13061, 13079, 13123,
       13149, 13187, 13193, 13197, 13215, 13225, 13249, 13281, 13289,
       13301, 16003, 16007, 16009, 16013, 16017, 16021, 16023, 16025,
       16035, 16037, 16043, 16045, 16049, 16053, 16057, 16059, 16061,
       16063, 16073, 16075, 16077, 16079, 16085, 16087, 17069, 17151,
       18013, 18115, 21139, 21143, 21201, 21225, 22023, 22087, 22123,
       25019, 26011, 26017, 26039, 26051, 26079, 26083, 26085, 26113,
       26119, 26133, 26155, 28013, 28019, 28043, 28057, 28065, 28155,
       29057, 29227,

In [83]:
indix = list(FDIC_p.index)
c_index = 0
pIndex = [x for x in indix if x[c_index] in problematic_c]
FDIC_sb_p = FDIC_p.loc[pIndex]
FDIC_sb_p

percap_pers_inc    jobs  \
FIPS_county county_state   year cpi_2018                              
1029        Cleburne, AL   1994 58.883015           16566.0  4730.0   
                           1995 60.674587           17255.0  4815.0   
                           1996 62.386536           17112.0  4800.0   
                           1997 63.779980           17235.0  4945.0   
                           1998 64.815109           18477.0  5027.0   
...                                                     ...     ...   
55051       Iron, WI       2016 95.607864           46966.0  2635.0   
                           2017 97.214310           48743.0  2594.0   
                           2018 100.000000          51471.0  2591.0   
55078       Menominee, WI* 1996 62.386536           14717.0  2174.0   
                           1997 63.779980           15653.0  2415.0   

                                            nonfarm_inc  nonfarm_jobs  \
FIPS_county county_state   year cpi_2018                                
1029        Cleburne, AL   1994 58.883015       11650.0         899.0   
                           1995 60.674587       13961.0        1015.0   
                           1996 62.386536       14896.0        1092.0   
                           1997 63.779980       16566.0        1123.0   
                           1998 64.815109       17401.0        1154.0   
...                                                 ...           ...   
55051       Iron, WI       2016 95.607864       19436.0         784.0   
                           2017 97.214310       18705.0         756.0   
                           2018 100.000000      19449.0         787.0   
55078       Menominee, WI* 1996 62.386536        3346.0         225.0   
                           1997 63.779980        3239.0         253.0   

                                            owner_inc  owner_jobs   cb  \
FIPS_county county_state   year cpi_2018                                 
1029        Cleburne, AL   1994 58.883015     32318.0      1329.0  0.0   
                           1995 60.674587     31914.0      1437.0  0.0   
                           1996 62.386536     27077.0      1501.0  0.0   
                           1997 63.779980     28388.0      1527.0  0.0   
                           1998 64.815109     32538.0      1568.0  0.0   
...                                               ...         ...  ...   
55051       Iron, WI       2016 95.607864     19753.0       833.0  0.5   
                           2017 97.214310     18981.0       804.0  0.5   
                           2018 100.000000    19459.0       835.0  0.5   
55078       Menominee, WI* 1996 62.386536      3346.0       225.0  0.0   
                           1997 63.779980      3239.0       253.0  0.0   

                                                pop  
FIPS_county county_state   year cpi_2018             
1029        Cleburne, AL   1994 58.883015   13030.0  
                           1995 60.674587   13080.0  
                           1996 62.386536   13254.0  
                           1997 63.779980   13768.0  
                           1998 64.815109   13921.0  
...                                             ...  
55051       Iron, WI       2016 95.607864    5650.0  
                           2017 97.214310    5672.0  
                           2018 100.000000   5676.0  
55078       Menominee, WI* 1996 62.386536    4555.0  
                           1997 63.779980    4592.0  

[7008 rows x 8 columns]

In [84]:
FDIC_sb_p['cb'] = [0 if x == 0 else 1 for x in list(FDIC_sb_p['cb'])]

In [85]:
FDIC_sb_p.to_csv("test.csv")